In [3]:
import os
import random
import imutils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.preprocessing import image
from PIL import Image, ImageDraw, ImageOps
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout
from keras.optimizers import Adam
from keras.layers.convolutional import Convolution2D, MaxPooling2D
#from keras.applications import VGG16

# seed для повторяемости результатов
np.random.seed(42)

In [4]:
X_train = np.load('/media/vlados/FreeSpace/Kaggle/dataset_2/X_train.npy')
Y_train = np.load('/media/vlados/FreeSpace/Kaggle/dataset_2/Y_train.npy')
x_test = np.load('/media/vlados/FreeSpace/Kaggle/dataset_2/x_test.npy')
y_test = np.load('/media/vlados/FreeSpace/Kaggle/dataset_2/y_test.npy')

In [5]:
x_test = x_test.astype('float16')
x_test /= 255

y_test = np_utils.to_categorical(y_test, 340)

In [6]:
X_train = X_train.astype('float16')
X_train /= 255

Y_train = np_utils.to_categorical(Y_train, 340)

In [17]:
model = Sequential()
model.add(Convolution2D(filters = 128, kernel_size = (3, 3), padding = 'same',
                 input_shape = (32,32,1), activation = 'relu', 
                 data_format = "channels_last"))
model.add(MaxPooling2D(pool_size = (2,2), 
                       data_format = "channels_last"))
model.add(Convolution2D(filters = 64, kernel_size = (3, 3), padding = 'valid',
                 activation = 'relu', data_format = "channels_last"))
model.add(MaxPooling2D(pool_size = (2,2), 
                       data_format = "channels_last"))
model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
#model.add(Dense(420, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(340, activation = 'softmax'))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 128)       1280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        73792     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              3212288   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
__________

In [18]:
model.compile(loss='categorical_crossentropy',
              optimizer='SGD', 
              metrics=['accuracy'])

In [19]:
model.fit(X_train, Y_train, epochs=3, validation_split=0.2, shuffle=True)

Train on 408000 samples, validate on 102000 samples
Epoch 1/3
408000/408000 [==============================] - 1045s 3ms/step - loss: 4.6236 - acc: 0.1231 - val_loss: 3.3723 - val_acc: 0.2919
Epoch 2/3
408000/408000 [==============================] - 1091s 3ms/step - loss: 3.0220 - acc: 0.3464 - val_loss: 2.5568 - val_acc: 0.4347
Epoch 3/3
408000/408000 [==============================] - 1145s 3ms/step - loss: 2.5011 - acc: 0.4388 - val_loss: 2.2772 - val_acc: 0.4889


In [39]:
scores = model.evaluate(x_test, y_test)

68000/68000 [==============================] - 52s 760us/step


model_json = model.to_json()

json_file = open('/media/vlados/FreeSpace/Kaggle/dataset/quick_doodle.h5', "w")
json_file.write(model_json)
json_file.close()

In [10]:
loaded_model.save_weights('/media/vlados/FreeSpace/Kaggle/dataset/quick_doodle.h5')

In [7]:
from keras.models import model_from_json

json_file = open("/media/vlados/FreeSpace/Kaggle/dataset/quick_doodle.json", "r")
loaded_model_json = json_file.read()
json_file.close()
# Создаем модель на основе загруженных данных
loaded_model = model_from_json(loaded_model_json)
# Загружаем веса в модель
loaded_model.load_weights('/media/vlados/FreeSpace/Kaggle/dataset/quick_doodle.h5')

In [8]:
loaded_model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

loaded_model.fit(X_train, Y_train, epochs=10, validation_split=0.2, shuffle=True)

scores = loaded_model.evaluate(x_test, y_test, verbose=0)
print("Точность модели на тестовых данных: %.2f%%" % (scores[1]*100))

Train on 408000 samples, validate on 102000 samples
Epoch 1/10
408000/408000 [==============================] - 990s 2ms/step - loss: 1.9341 - acc: 0.5519 - val_loss: 1.7235 - val_acc: 0.5969
Epoch 2/10
408000/408000 [==============================] - 934s 2ms/step - loss: 1.7667 - acc: 0.5850 - val_loss: 1.6817 - val_acc: 0.6052
Epoch 3/10
408000/408000 [==============================] - 918s 2ms/step - loss: 1.6722 - acc: 0.6034 - val_loss: 1.6612 - val_acc: 0.6090
Epoch 4/10
408000/408000 [==============================] - 996s 2ms/step - loss: 1.5910 - acc: 0.6203 - val_loss: 1.6622 - val_acc: 0.6078
Epoch 5/10
408000/408000 [==============================] - 987s 2ms/step - loss: 1.5238 - acc: 0.6322 - val_loss: 1.6468 - val_acc: 0.6121
Epoch 6/10
408000/408000 [==============================] - 981s 2ms/step - loss: 1.4600 - acc: 0.6448 - val_loss: 1.6397 - val_acc: 0.6146
Epoch 7/10
408000/408000 [==============================] - 984s 2ms/step - loss: 1.4011 - acc: 0.6569 - val

In [ ]:
model_json = loaded_model.to_json()
# Записываем модель в файл
json_file = open("/media/vlados/FreeSpace/Kaggle/dataset/quick_doodle.json", "w")
json_file.write(model_json)
json_file.close()